In [1]:
from dotenv import load_dotenv
import robin_stocks.robinhood as rs
import os
import requests
import time
import json

## Global To-do

1. Setup a ledger for when trades are made with which stock made the trade
2. Check for issues in Stock class
3. update data distribution with clause for if data doesn't come in
4. Inquire/study for a better equation for reserve_cash distribution

## Initialization
    Functions used in Staging Area

In [2]:
def rh_login():
    load_dotenv()
    robin_user = os.environ.get("robinhood_username")
    robin_pass = os.environ.get("robinhood_password")
    rs.login(username=robin_user,
             password=robin_pass,
             expiresIn=86400,
             by_sms=True)

In [3]:
def time_wizard(time_count):
    time_count += 1
    if time_count == 6:
        time_count = 1
    #print(f"Time Count: {time_count}")
    return time_count

In [4]:
def get_top_100():
    top100_raw_dict = rs.get_top_100()
    top_100_file_path= "./Top_100.json"
    with open(top_100_file_path, "w+") as file:
        json.dump(top100_raw_dict, file)
    #print("updating raw")
    return(top100_raw_dict)

In [5]:
def get_money_in_account():
    global reserve_cash
    profile_info = rs.build_user_profile()
    reserve_cash = float(profile_info['cash'])-25000 #to account for buy-in money that cannot be traded

In [6]:
def set_buy_ratio():
    global positive_sin_value
    global reserve_cash
    value_list = []
    
    for stock_class in positive_sin_value:
        value_list.append(stock_class.sin_factor)
    #print(f"value_list: {value_list}")
        
    average_sin = sum(value_list) / len(value_list)
    #print(f"average_sin: {average_sin}")
    
    for stock_class in positive_sin_value:
        print(f"((({stock_class.sin_factor}/{average_sin})*{reserve_cash})/{len(value_list)})*{stock_class.sin_factor}")
        stock_class.amount_to_spend = (((stock_class.sin_factor/average_sin)*reserve_cash)/len(value_list)*stock_class.sin_factor)
       # print(stock_class.amount_to_spend)
        
    positive_sin_value = []
        
        
    
        

## Data Distribution
    receives data from get_top_100()
    makes stock classes if no stocks in list
    looks for stock symbol in stock class and will update each class if they are in stocks_list
    
    *Need to delete stock price history data if not on list* good to keep the class until EoD anyway

In [7]:
def update_stock_classes(top100_raw_dict):
    global stocks_list
    for stock_dict in top100_raw_dict:
        if stock_dict["symbol"] not in stocks_list:
            stock_class = Stock(stock_dict)

    #print(stocks_list)         
        else:
            for stock_name, stock_class in stocks_list.items():
                if stock_name == stock_dict["symbol"]: 
                    stock_class.ask_price = stock_dict.get("ask_price")
                    stock_class.update()
                    stock_class.print_variable()
            
    

## Stock Class
    When class is created all stock data is turned into class variables (Thank you Julian)
    class is added key, value as symbol, class to the stocks_list dictionary
    update method is invoked from update_stock_classes()
    update method includes the equation_of_sin() method
    
    Need to:
        
        Add sell button that would sell the stock if sin_factor was less than 0 or if it was 30 minutes before closing
        Add equation to calculate profit or loss from buy time to sell time
        
        
        
        

In [8]:
class Stock:
    #Name
    def __str__(self):
        return self.symbol

    #Enter the Battlefield
    def __init__(self, stock_dict):
        # set the input items as class members
        for key, value in stock_dict.items():
            exec("self." + key + " = value")
            
    #Key Words
        self.stock_dict = stock_dict
        stocks_list[self.symbol] = self #add to dictionary with key as symbol, and value as the class
        self.state = "UNBOUGHT"
        self.shares = 0
        self.price_history=[]
        self.sin_factor = None
        self.amount_to_spend = None
        self.price_history.append(self.ask_price)
        
        
        
    #Abilities
   
    def update(self):
        self.price_history.append(self.ask_price)
        
        #makes the program have at least 5 data points before moving to the next step
        if len(self.price_history) >= 6:
            self.price_history.pop(0)
        else:
            pass
        

        
        if len(self.price_history) >= 5:
            self.equation_of_sin()
            
        if self.amount_to_spend is not None:
            self.buy()
        
        if self.sin_factor is not None and self.sin_factor < 0:
            self.sell()
        
        
    
    def equation_of_sin(self):
        #pythagorean theorem
        self.a = len(self.price_history)**2
        self.price_differential = float(self.price_history[-1]) - float(self.price_history[0])
        self.b = self.price_differential**2
        self.c = math.sqrt(self.a + self.b)
        
        self.sin_factor = round((self.b / self.c), 2)
        
        if self.sin_factor > 0:
            positive_sin_value.append(self)
            
    def buy(self):
        global reserve_cash
        amount_you_can_buy = float(self.amount_to_spend)/float(self.ask_price)
        amount_rounded = round(amount_you_can_buy)
        if amount_rounded < 1:
            pass
        else:
            print(f"I am buying {amount_rounded} of {self.symbol}")
            reserve_cash = reserve_cash - self.amount_to_spend
            self.shares += amount_rounded
            
        self.amount_to_spend = None
        
    def sell(self):
        global reserve_cash
        sell_value = self.ask_price * self.shares
        print(f"I am selling {self.shares} of {self.symbol}")
        self.shares = 0
        reserve_cash = reserve_cash + sell_value
        
        

    def print_variable(self):
        print(f"{self.symbol} : {self.price_history}")
        if self.amount_to_spend is not None:
            print(f"{self.symbol}: {self.amount_to_spend}")
    

## Staging Area
    Sets Modules
    Sets Global variables
    Keeps time using time_wizard()
    logs into Robinhood using rh_login()
    pulls buying power info from account with get_money_in_account()
    pulls stock data from API using get_top_100()
    stock data goes into update_stock_classes()
    when stocks show positive sin_value, it distributes reserve cash to the stock class for calulating buying

In [9]:
import os
import robin_stocks.robinhood as rs
from dotenv import load_dotenv
import json
import math

#Pre-Game Actions/Values
rh_login()
time_count = 0
gains_losses = {}
stocks_list = {}
reserve_cash = float(30000)
positive_sin_value = []

#Main Phase
while True: 
    print(f"TICK: {time_count} \n ----------------------------------------------------------------------------------------------------------------")
    time_count = time_wizard(time_count)
    #get_money_in_account()
    update_stock_classes(get_top_100())
    
    print(positive_sin_value)
    if positive_sin_value != []:
        set_buy_ratio()
    
        
    
    time.sleep(5)

TICK: 0 
 ----------------------------------------------------------------------------------------------------------------
[]
I pass the turn
TICK: 1 
 ----------------------------------------------------------------------------------------------------------------
TSLA : ['239.630000', '239.630000']
AAPL : ['185.370000', '185.370000']
AMZN : ['128.010000', '128.010000']
F : ['14.200000', '14.200000']
AMC : ['4.050000', '4.050000']
APE : ['1.800000', '1.800000']
MSFT : ['329.260000', '329.260000']
SNDL : ['1.310000', '1.310000']
DIS : ['88.940000', '88.940000']
NIO : ['8.420000', '8.420000']
META : ['279.450000', '279.450000']
LCID : ['5.750000', '5.750000']
NVDA : ['405.930000', '405.930000']
VOO : ['398.520000', '398.520000']
NFLX : ['415.570000', '415.570000']
SPY : ['432.040000', '432.040000']
GOOGL : ['118.590000', '118.590000']
SNAP : ['11.220000', '11.220000']
GPRO : ['4.140000', '4.140000']
PFE : ['36.880000', '36.880000']
BABA : ['85.190000', '85.190000']
PLUG : ['9.170000', '9

TICK: 3 
 ----------------------------------------------------------------------------------------------------------------
TSLA : ['239.630000', '239.630000', '239.600000', '239.600000']
AAPL : ['185.370000', '185.370000', '185.370000', '185.370000']
AMZN : ['128.010000', '128.010000', '128.010000', '128.010000']
F : ['14.200000', '14.200000', '14.200000', '14.200000']
AMC : ['4.050000', '4.050000', '4.050000', '4.050000']
APE : ['1.800000', '1.800000', '1.800000', '1.800000']
MSFT : ['329.260000', '329.260000', '329.260000', '329.260000']
SNDL : ['1.310000', '1.310000', '1.310000', '1.310000']
DIS : ['88.940000', '88.940000', '88.940000', '88.940000']
NIO : ['8.420000', '8.420000', '8.420000', '8.420000']
META : ['279.450000', '279.450000', '279.450000', '279.450000']
LCID : ['5.750000', '5.750000', '5.750000', '5.750000']
NVDA : ['405.930000', '405.930000', '405.930000', '405.930000']
VOO : ['398.520000', '398.520000', '398.520000', '398.520000']
NFLX : ['415.570000', '415.570000', '

TICK: 5 
 ----------------------------------------------------------------------------------------------------------------
TSLA : ['239.630000', '239.600000', '239.600000', '239.600000', '239.550000']
AAPL : ['185.370000', '185.370000', '185.370000', '185.370000', '185.420000']
AMZN : ['128.010000', '128.010000', '128.010000', '128.010000', '128.010000']
F : ['14.200000', '14.200000', '14.200000', '14.200000', '14.200000']
AMC : ['4.050000', '4.050000', '4.050000', '4.050000', '4.050000']
APE : ['1.800000', '1.800000', '1.800000', '1.800000', '1.800000']
MSFT : ['329.260000', '329.260000', '329.260000', '329.260000', '329.420000']
SNDL : ['1.310000', '1.310000', '1.310000', '1.310000', '1.310000']
DIS : ['88.940000', '88.940000', '88.940000', '88.940000', '88.940000']
NIO : ['8.420000', '8.420000', '8.420000', '8.420000', '8.420000']
META : ['279.450000', '279.450000', '279.450000', '279.450000', '279.450000']
LCID : ['5.750000', '5.750000', '5.750000', '5.750000', '5.750000']
NVDA : [

KeyboardInterrupt: 

## Testing Area

##  Dictionary/List Reminders
positive_sin_value = classes with positive sin value

stocks_list (x:y), x= stock symbol, y=associated stock class

stockdict, check Top_100.json

## This to change before Release
1. set reserve_cash to pull from the server, not be set like for testing
        -need to choose between pulling reserve cash only once at beginning of day or possibly repulling after a successful sell to basically do a doublecheck
        
2.Need to set actual buy and sell methods to interact with RH

## Equation for dividing available money between available stocks
s = personal stock sin
a = average stock sin
m = money in pool
n = number of stocks with a positive sin

(((s/a)*m)/n)s